# Consolidación de Panel Maestro UPV

En este notebook vamos a unificar los 4 datasets panel en uno solo:
- **Satisfacción**: Valoraciones de alumnos y profesores
- **Abandono**: Tasas de abandono y permanencia
- **Autoeficacia**: Autoeficacia percibida de 3 años
- **Empleabilidad**: Porcentaje de no desempleados

El resultado será un CSV maestro completo con todas las métricas unificadas.

## 1. Cargar todos los datasets panel

In [ ]:
import pandas as pd
import numpy as np

# Cargar todos los datasets panel
panel_abandono = pd.read_csv('abandono/panel_abandono_UPV.csv', encoding='utf-8')
panel_autoeficacia = pd.read_csv('autoeficacia/panel_autoeficacia_UPV.csv', encoding='utf-8')
panel_no_desempleados = pd.read_csv('no_desempleados/panel_no_desempleados_UPV.csv', encoding='utf-8')
panel_satisfaccion = pd.read_csv('satisfaccion/panel_satisfaccion_UPV.csv', encoding='utf-8')

print("✅ Todos los datasets cargados correctamente\n")
print(f"Abandono: {panel_abandono.shape}")
print(f"Autoeficacia: {panel_autoeficacia.shape}")
print(f"No desempleados: {panel_no_desempleados.shape}")
print(f"Satisfacción: {panel_satisfaccion.shape}")

print("\n=== PRIMERAS FILAS DE CADA DATASET ===")
print("\nAbandonoñ:")
print(panel_abandono.head(2))
print("\nAutoeficacia:")
print(panel_autoeficacia.head(2))
print("\nNo desempleados:")
print(panel_no_desempleados.head(2))
print("\nSatisfacción:")
print(panel_satisfaccion.head(2))

## 2. Explorar estructura e identificar claves comunes

In [ ]:
print("=== ESTRUCTURA DE CADA DATASET ===")

print("\n--- ABANDONO ---")
print(f"Columnas: {list(panel_abandono.columns)}")
print(f"Tipos: {dict(panel_abandono.dtypes)}")

print("\n--- AUTOEFICACIA ---")
print(f"Columnas: {list(panel_autoeficacia.columns)}")
print(f"Tipos: {dict(panel_autoeficacia.dtypes)}")

print("\n--- NO DESEMPLEADOS ---")
print(f"Columnas: {list(panel_no_desempleados.columns)}")
print(f"Tipos: {dict(panel_no_desempleados.dtypes)}")

print("\n--- SATISFACCIÓN ---")
print(f"Columnas: {list(panel_satisfaccion.columns)}")
print(f"Tipos: {dict(panel_satisfaccion.dtypes)}")

# Identificar columnas comunes
print("\n=== ANÁLISIS DE COLUMNAS COMUNES ===")
cols_abandono = set(panel_abandono.columns)
cols_autoeficacia = set(panel_autoeficacia.columns)
cols_no_desempleados = set(panel_no_desempleados.columns)
cols_satisfaccion = set(panel_satisfaccion.columns)

# Intersección de todas
comunes = cols_abandono.intersection(cols_autoeficacia, cols_no_desempleados, cols_satisfaccion)
print(f"Columnas comunes a todos: {sorted(comunes)}")

# Claves de merge
claves_merge = ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'año']
print(f"\nClaves de merge propuestas: {claves_merge}")
print(f"¿Están todas en cada dataset?")
for clave in claves_merge:
    print(f"  {clave}: Abandono={clave in cols_abandono}, Autoeficacia={clave in cols_autoeficacia}, NoDesempleados={clave in cols_no_desempleados}, Satisfacción={clave in cols_satisfaccion}")

## 3. Estandarizar nombres de columnas y tipos de datos

In [ ]:
# Crear copias de los datasets
df_abandono = panel_abandono.copy()
df_autoeficacia = panel_autoeficacia.copy()
df_no_desempleados = panel_no_desempleados.copy()
df_satisfaccion = panel_satisfaccion.copy()

print("=== ESTANDARIZACIÓN DE DATOS ===")

# Asegurar que las claves sean del mismo tipo
for df in [df_abandono, df_autoeficacia, df_no_desempleados, df_satisfaccion]:
    df['año'] = df['año'].astype(int)
    df['COD_RUCT'] = df['COD_RUCT'].astype(int)
    df['CURSO'] = df['CURSO'].astype(str)
    df['TITULACION'] = df['TITULACION'].astype(str)
    df['CENTRO'] = df['CENTRO'].astype(str)

print("✅ Tipos de datos estandarizados en las claves de merge")
print(f"\nTipos en abandono: {dict(df_abandono[['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'año']].dtypes)}")

# Renombrar columnas para evitar conflictos
# Las columnas específicas de cada dataset mantendrán sus nombres
print("\n✅ Estructura lista para merge")

## 4. Fusionar datasets en las claves comunes

In [ ]:
# Definir claves de merge
claves = ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'año']

print("=== PROCESO DE FUSIÓN ===")
print(f"Claves de merge: {claves}\n")

# Paso 1: Empezar con satisfacción como base
print("Paso 1: Base = Satisfacción")
panel_maestro = df_satisfaccion.copy()
print(f"  Shape: {panel_maestro.shape}")
print(f"  Columnas: {list(panel_maestro.columns)}")

# Paso 2: Merge con Abandono
print("\nPaso 2: Merge con Abandono (LEFT JOIN)")
columns_antes = len(panel_maestro.columns)
panel_maestro = panel_maestro.merge(
    df_abandono,
    on=claves,
    how='left',
    validate='m:m'
)
print(f"  Shape después: {panel_maestro.shape}")
print(f"  Nuevas columnas: {len(panel_maestro.columns) - columns_antes}")

# Paso 3: Merge con Autoeficacia
print("\nPaso 3: Merge con Autoeficacia (LEFT JOIN)")
columns_antes = len(panel_maestro.columns)
panel_maestro = panel_maestro.merge(
    df_autoeficacia,
    on=claves,
    how='left',
    validate='m:m'
)
print(f"  Shape después: {panel_maestro.shape}")
print(f"  Nuevas columnas: {len(panel_maestro.columns) - columns_antes}")

# Paso 4: Merge con No Desempleados
print("\nPaso 4: Merge con No Desempleados (LEFT JOIN)")
columns_antes = len(panel_maestro.columns)
panel_maestro = panel_maestro.merge(
    df_no_desempleados,
    on=claves,
    how='left',
    validate='m:m'
)
print(f"  Shape después: {panel_maestro.shape}")
print(f"  Nuevas columnas: {len(panel_maestro.columns) - columns_antes}")

print(f"\n✅ Merge completado")
print(f"\nDataset maestro final:")
print(f"  Filas: {len(panel_maestro)}")
print(f"  Columnas: {len(panel_maestro.columns)}")
print(f"  Columnas totales: {list(panel_maestro.columns)}")

## 5. Consolidar columnas duplicadas y organizar

In [ ]:
print("=== ANÁLISIS DE COLUMNAS DUPLICADAS ===")

# Verificar si hay sufijos (_x, _y) que indiquen duplicados
duplicadas = [col for col in panel_maestro.columns if col.endswith('_x') or col.endswith('_y')]
print(f"Columnas con sufijo _x o _y: {duplicadas}")

if duplicadas:
    print("\nConsolidando columnas duplicadas...")
    for col_x in [col for col in duplicadas if col.endswith('_x')]:
        col_y = col_x.replace('_x', '_y')
        col_base = col_x.replace('_x', '')
        if col_y in panel_maestro.columns:
            # Verificar si son idénticas
            if panel_maestro[col_x].equals(panel_maestro[col_y]):
                print(f"  {col_base}: columnas idénticas, manteniendo _x")
                panel_maestro[col_base] = panel_maestro[col_x]
                panel_maestro = panel_maestro.drop([col_x, col_y], axis=1)
            else:
                print(f"  {col_base}: columnas diferentes, verificando valores faltantes")
                # Combinar: si _x tiene NaN, usar _y
                panel_maestro[col_base] = panel_maestro[col_x].fillna(panel_maestro[col_y])
                panel_maestro = panel_maestro.drop([col_x, col_y], axis=1)

print("\n✅ Columnas consolidadas")
print(f"\nColumnas finales ({len(panel_maestro.columns)}):")
for i, col in enumerate(panel_maestro.columns, 1):
    print(f"  {i:2d}. {col}")

## 6. Realizar verificaciones de calidad de datos

In [ ]:
print("=== VERIFICACIONES DE CALIDAD DE DATOS ===")

# 1. Valores faltantes
print("\n1. VALORES FALTANTES POR COLUMNA:")
missing = panel_maestro.isnull().sum()
missing_pct = (missing / len(panel_maestro) * 100).round(2)
for col in panel_maestro.columns:
    if missing[col] > 0:
        print(f"  {col}: {missing[col]} ({missing_pct[col]}%)")
print(f"  Total sin valores faltantes: {(missing == 0).sum()} columnas")

# 2. Duplicados
print("\n2. DUPLICADOS:")
duplicated_rows = panel_maestro.duplicated(subset=None, keep=False).sum()
print(f"  Filas completamente duplicadas: {duplicated_rows}")

# 3. Filas por año
print("\n3. DISTRIBUCIÓN POR AÑO:")
print(panel_maestro['año'].value_counts().sort_index())

# 4. Institutos/Centros
print("\n4. COBERTURA DE DATOS:")
print(f"  Años únicos: {sorted(panel_maestro['año'].unique())}")
print(f"  Centros únicos: {panel_maestro['CENTRO'].nunique()}")
print(f"  Titulaciones únicas: {panel_maestro['TITULACION'].nunique()}")
print(f"  Observaciones totales: {len(panel_maestro)}")

# 5. Verificar rangos de valores numéricos
print("\n5. RANGOS DE VALORES NUMÉRICOS:")
numeric_cols = panel_maestro.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    non_null = panel_maestro[col].dropna()
    if len(non_null) > 0:
        print(f"  {col}: min={non_null.min():.2f}, max={non_null.max():.2f}, media={non_null.mean():.2f}")

# 6. Tipos de datos
print("\n6. TIPOS DE DATOS:")
print(panel_maestro.dtypes)

## 7. Reorganizar columnas de forma lógica

In [ ]:
# Reorganizar columnas de forma lógica
print("=== REORGANIZACIÓN DE COLUMNAS ===")

# Definir orden de columnas
# 1. Identificadores
cols_identificadores = ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'año']

# 2. Satisfacción
cols_satisfaccion = [col for col in panel_maestro.columns if 'satisfaccion' in col.lower() or 'diferencia_satis' in col.lower()]

# 3. Abandono
cols_abandono = [col for col in panel_maestro.columns if 'abandono' in col.lower() or 'permanencia' in col.lower()]

# 4. Autoeficacia
cols_autoeficacia = [col for col in panel_maestro.columns if 'autoeficacia' in col.lower() or 'nivel_autoeficacia' in col.lower()]

# 5. Empleabilidad
cols_empleabilidad = [col for col in panel_maestro.columns if 'desempleado' in col.lower() or 'empleabilidad' in col.lower()]

# Crear orden final
cols_ordenadas = (
    cols_identificadores +
    cols_satisfaccion +
    cols_abandono +
    cols_autoeficacia +
    cols_empleabilidad
)

# Verificar que se incluyen todas las columnas
columnas_faltantes = set(panel_maestro.columns) - set(cols_ordenadas)
if columnas_faltantes:
    print(f"Columnas no clasificadas: {columnas_faltantes}")
    cols_ordenadas.extend(list(columnas_faltantes))

# Reorganizar
panel_maestro = panel_maestro[cols_ordenadas]

print(f"\nOrden final de columnas ({len(panel_maestro.columns)}):")
print("\n--- Identificadores ---")
for col in cols_identificadores:
    if col in panel_maestro.columns:
        print(f"  {col}")

print("\n--- Satisfacción ---")
for col in cols_satisfaccion:
    if col in panel_maestro.columns:
        print(f"  {col}")

print("\n--- Abandono ---")
for col in cols_abandono:
    if col in panel_maestro.columns:
        print(f"  {col}")

print("\n--- Autoeficacia ---")
for col in cols_autoeficacia:
    if col in panel_maestro.columns:
        print(f"  {col}")

print("\n--- Empleabilidad ---")
for col in cols_empleabilidad:
    if col in panel_maestro.columns:
        print(f"  {col}")

if columnas_faltantes:
    print(f"\n--- Otras ---")
    for col in columnas_faltantes:
        print(f"  {col}")

print(f"\n✅ Columnas reorganizadas")

## 8. Exportar dataset maestro

In [ ]:
# Guardar el dataset maestro
archivo_salida = 'panel_maestro_UPV.csv'
panel_maestro.to_csv(archivo_salida, index=False, encoding='utf-8')

print(f"✅ Dataset maestro guardado como '{archivo_salida}'")
print(f"\nDimensiones finales:")
print(f"  Filas: {len(panel_maestro):,}")
print(f"  Columnas: {len(panel_maestro.columns)}")
print(f"\nPrimeras filas del dataset final:")
print(panel_maestro.head())
print(f"\nÚltimas filas del dataset final:")
print(panel_maestro.tail())

## 9. Resumen final y diccionario de datos

In [ ]:
print("=" * 80)
print("RESUMEN DEL PANEL MAESTRO UPV")
print("=" * 80)

print(f"\n📊 DIMENSIONES:")
print(f"  Total de observaciones: {len(panel_maestro):,}")
print(f"  Total de variables: {len(panel_maestro.columns)}")
print(f"  Años cubiertos: {sorted(panel_maestro['año'].unique())}")
print(f"  Rango temporal: {panel_maestro['año'].min()}-{panel_maestro['año'].max()}")

print(f"\n🏛️ COBERTURA INSTITUCIONAL:")
print(f"  Centros únicos: {panel_maestro['CENTRO'].nunique()}")
print(f"  Titulaciones únicas: {panel_maestro['TITULACION'].nunique()}")
print(f"  Códigos RUCT únicos: {panel_maestro['COD_RUCT'].nunique()}")

print(f"\n📋 DICCIONARIO DE DATOS:")
print(f"\nIden tificadores:")
for col in cols_identificadores:
    if col in panel_maestro.columns:
        dtype = panel_maestro[col].dtype
        unique = panel_maestro[col].nunique()
        missing = panel_maestro[col].isnull().sum()
        print(f"  • {col:40s} | Tipo: {str(dtype):10s} | Únicos: {unique:4d} | Nulos: {missing:4d}")

print(f"\nSatisfacción (Escala 0-10):")
for col in cols_satisfaccion:
    if col in panel_maestro.columns:
        dtype = panel_maestro[col].dtype
        unique = panel_maestro[col].nunique()
        missing = panel_maestro[col].isnull().sum()
        print(f"  • {col:40s} | Tipo: {str(dtype):10s} | Únicos: {unique:4d} | Nulos: {missing:4d}")

print(f"\nAbandonoñ:")
for col in cols_abandono:
    if col in panel_maestro.columns:
        dtype = panel_maestro[col].dtype
        unique = panel_maestro[col].nunique()
        missing = panel_maestro[col].isnull().sum()
        print(f"  • {col:40s} | Tipo: {str(dtype):10s} | Únicos: {unique:4d} | Nulos: {missing:4d}")

print(f"\nAutoeficacia:")
for col in cols_autoeficacia:
    if col in panel_maestro.columns:
        dtype = panel_maestro[col].dtype
        unique = panel_maestro[col].nunique()
        missing = panel_maestro[col].isnull().sum()
        print(f"  • {col:40s} | Tipo: {str(dtype):10s} | Únicos: {unique:4d} | Nulos: {missing:4d}")

print(f"\nEmpleabilidad:")
for col in cols_empleabilidad:
    if col in panel_maestro.columns:
        dtype = panel_maestro[col].dtype
        unique = panel_maestro[col].nunique()
        missing = panel_maestro[col].isnull().sum()
        print(f"  • {col:40s} | Tipo: {str(dtype):10s} | Únicos: {unique:4d} | Nulos: {missing:4d}")

print(f"\n✅ Panel maestro consolidado correctamente")
print(f"\nArchivo guardado: panel_maestro_UPV.csv")